# Reproduction table 1

In this notebook, we generate a glycogen structure and we calculate some structural properties

In [1]:
import math
import numpy as np
from pylab import *
import os
import os.path
import json
import glycogen_module as gm
import random
import pandas as pd
import matplotlib.pyplot as plt

random.seed(123)  # add seed for reproducibility

with open("parameters.json", "r") as f:
    parameters = json.load(f)


In [2]:
from algorithm import Gillespie_step


## Generating a granule

In [3]:
my_granule = gm.glycogen_structure(parameters, 2, 1)

parameters['number_max_of_monomer'] = 2000

N = my_granule.number_of_glucose_fixed()

# here set GS = 1.0 and GBE = 0.1 if you want to produce the figure for gamma = 10.0
C0 = {"GS": 0.2, "GBE": 1, "GP": 0.0, "GDE": 0.0}
k = {"k_GS": 1, "k_GBE": 1, "k_GP": 1, "k_GDE": 1}

while N < parameters['number_max_of_monomer']:
    if N % 5000 == 0:
        print('avancement = ', 100*N /
              parameters['number_max_of_monomer'], '%', '   part 1/2')
    reaction, time = Gillespie_step(my_granule, C0, k)

    if reaction == 'Act_gs()':
        my_granule.Act_gs()
    elif reaction == 'Act_gp()':
        my_granule.Act_gp()
    elif reaction == 'Act_gbe()':
        my_granule.Act_gbe_flexible_model()
    elif reaction == 'Act_gde()':
        my_granule.Act_gde()

    N = my_granule.number_of_glucose_fixed()

# write output
name = 'glycogen_structure_N='+str(N)+'_gamma='+str(C0['GS']/C0['GBE'])+'.json'
my_granule.write_structure(name)


## Structural properties calculation

In [4]:
print('Number of non-reducing ends (nre):', my_granule.number_of_chain())
print('A:B ratio:', my_granule.AtoBratio())
print('Average chain length:', my_granule.avg_cl())
print('Branching degree:', my_granule.bd())
print('Last generation:', my_granule.get_last_gen_index())
print('Occupancy:', my_granule.occupancy())

r_g, _ = my_granule.get_radius('nm')
print('Radius of gyration:', r_g, 'nm', 'for', N, 'glucose units')


Number of non-reducing ends (nre): 234
A:B ratio: 0.828125
Average chain length: 8.547008547008547
Branching degree: 0.1319365798414496
Last generation: 9
Occupancy: 0.26501378402271314
Radius of gyration: 8.315951260110426 nm for 2000 glucose units
